In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LambdaCallback
import numpy as np
import random
import sys
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

newcorpus = PlaintextCorpusReader('data', '.*')
files = newcorpus.fileids()
raw_text = ''

for f in files:
    #get raw text
    raw_text += newcorpus.raw(f)
    
text = raw_text.lower()
print('text length', len(text))
chars = sorted(list(set(text))) # getting all unique chars
print('total chars: ', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end = on_epoch_end)


filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)


callbacks = [print_callback, checkpoint, reduce_lr]

model.fit(x, y, batch_size=128, epochs=20, callbacks=callbacks)



text length 241726
total chars:  57
Epoch 1/20
80562/80562 [==============================] - 86s 1ms/step - loss: 2.0969

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "you a happy christmas. i like to think i"
you a happy christmas. i like to think in the war the from the comman the granders of the christmas the gration stre the great the grand and the comman who stre the react to stre the christmas and the great in the great the remont and a wish and the stre the christmas the reation of the grand and the christmas to all of the prostict the stre the remple the great the christmas the gration and the christmas and us the christmas to which t
----- diversity: 0.5
----- Generating with seed: "you a happy christmas. i like to think i"
you a happy christmas. i like to think in the friends to to not who dive in the frighis in it the from and the brimis and your of the cart the from at the grabled and the christmas tods a can we so man the brictid m

of the message which inspired christ's find the same as all the world to the people who great common tround the our common for the world to the present family and with christmas disurals and in the world in their own common these are people who changes and to self-divaled with drear the serve of the morted the common even their world to the future of the great and events of the common the common bring need in the fether and to religid t
----- diversity: 1.0
----- Generating with seed: "of the message which inspired christ's f"
of the message which inspired christ's for what from not and realise of all a my reconcilly teammonets or the common fill people face to make as all of the worlds meets otherss in trist come of ind they wan helpmory of them to renosged can seamen spety and cavemoht of them may our very loving peace and tecom theurantes have colading. fher help, unders, ghe or day igters aroutage and peace of the grandfating to their ipportand and realle
----- diversity: 1.2
-----

is anstrementing telliamtry.u
burly. yecrisppaces of yeainen held, it is lestonh.

i focousiti but  nos nown sersonntors. family, in pree tient in so those what follone whe, remains tealve commonwealth. who have - other havits which atsentements owither than was. but my family and dus to kneded these practivel, such blitiry, your the 

Epoch 00007: loss improved from 1.28355 to 1.25131, saving model to weights.hdf5
Epoch 8/20
80562/80562 [==============================] - 119s 1ms/step - loss: 1.2261

----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "e actual, birthday of a man who was dest"
e actual, birthday of a man who was destration to the commonwealth to the commonwealth christmas is a share and the commonwealth to the commonwealth to the commonwealth in the commonwealth in the commonwealth in the commonwealth to the commonwealth is a time of the commonwealth can be a prossers of the commonwealth christmas is a should be a millions of the comm

ic islands. but in fact they are all lint of the gatheroms long ablow that companions of the commonwealth. we can be common than the commonwealth came the commonwealth family and respect for the christian samper and the world and the commonwealth communities.

the grew whe community, young and i believe to no le to the world are place, and the commonwealth christmas day. their children and provided happy. we can refummer that we are com
----- diversity: 1.0
----- Generating with seed: "ic islands. but in fact they are all lin"
ic islands. but in fact they are all lin is sitcorgon us, i have scilie me a litcuris.

my hoage and the staid i am sure life christians, there is are this impriratits, the here of everywheness this journey ene being for the med, for many means char, is a mught and a family and gathernanre of service and bother into progress propeation of man ovent provided by have of us and it is strine and provided the hearticed their been dounding an
----- diversity: 1.2
-----

s of you expressed your grief most poigne and their wimm no a melss pable shipage and is a time for userilged and decince inford of the christmas entwheld what the gut lang new . has been leveliem in mana recuing christmas will rimal lintss chrict give; and undivents.

whattroaingent contrave that down to think most rayman.

love our years from pastd be green. 
 affacce, it, gratitudise influe, were real serviken what in tam particilarm

Epoch 00014: loss improved from 1.14146 to 1.13142, saving model to weights.hdf5
Epoch 15/20
80562/80562 [==============================] - 107s 1ms/step - loss: 1.1166

----- Generating text after Epoch: 14
----- diversity: 0.2
----- Generating with seed: "isbane for the commonwealth games and th"
isbane for the commonwealth games and the commonwealth countries and the commonwealth and person and the commonwealth and remind the commonwealth and in the commonwealth countries and the world which we should be a hand the commonwealth and the commonwealth 

ok place almost 2000 years ago; every year is not be asked and when the commonwealth in the world we with the world we should in the world we should in the world we should in the commonwealth in the world with the commonwealth in the world to the commonwealth for the commonwealth in the world we should in the world we should in the world when i was the commonwealth in the world, in the world we are able to the commonwealth is a time who
----- diversity: 0.5
----- Generating with seed: "ok place almost 2000 years ago; every ye"
ok place almost 2000 years ago; every year is the commonwealth and whet i make a family of our families. it can food and to come of the world to which the past and if was of the commonwealth countries to the future of the world to the birth of the commonwealth how one of the sacial of the world. i have self-continuing to make last people and help for them to create of our commonwealth have been the hearts and deepents of the family for 
----- diversity: 1.0
-----